<a href="https://colab.research.google.com/github/Jaeljane/Cab-Insight/blob/main/Insight_for_Cab_Investment_firm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install plotly

In [42]:
#Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly as py
import plotly.express as px
# import plotly.graph_objs as go
# from plotly.subplots import make_subplots
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

###**Import Data**
Import and Merge all the datasets to form one master Data

In [43]:
from datetime import datetime

def wrangle (w, x, y, z):
  #read in Cab data
  cab=pd.read_csv(w, parse_dates=True)
  #change Date of Travel data type to date time
  cab['Date of Travel']=cab['Date of Travel'].map(lambda date : datetime.fromordinal(datetime(1900,1,1).toordinal() + date))
  #read in City data
  city=pd.read_csv(x)
  #read in Customer Data
  customer=pd.read_csv(y)
  #Read in Transaction data
  Transaction=pd.read_csv(z)
  #Merge Cab and Transaction  Data
  cab_trans=cab.merge(Transaction, on='Transaction ID', how='left')
  #Merge Cab, Transaction and Customer Data sets
  data=cab_trans.merge(customer, on='Customer ID', how='left')
  #Merge Cab, Transaction, Customer with City Data
  data=data.merge(city, on="City", how="left")

  return data


In [44]:
cab_url='https://raw.githubusercontent.com/DataGlacier/DataSets/main/Cab_Data.csv'
city_url='https://raw.githubusercontent.com/DataGlacier/DataSets/main/City.csv'
customer_url='https://raw.githubusercontent.com/DataGlacier/DataSets/main/Customer_ID.csv'
transaction_url='https://raw.githubusercontent.com/DataGlacier/DataSets/main/Transaction_ID.csv'

df=wrangle(cab_url, city_url, customer_url, transaction_url)
df.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip,Customer ID,Payment_Mode,Gender,Age,Income (USD/Month),Population,Users
0,10000011,2016-01-10,Pink Cab,ATLANTA GA,30.45,370.95,313.635,29290,Card,Male,28,10813,"814,885","24,701"
1,10000012,2016-01-08,Pink Cab,ATLANTA GA,28.62,358.52,334.854,27703,Card,Male,27,9237,"814,885","24,701"
2,10000013,2016-01-04,Pink Cab,ATLANTA GA,9.04,125.20,97.632,28712,Cash,Male,53,11242,"814,885","24,701"
3,10000014,2016-01-09,Pink Cab,ATLANTA GA,33.17,377.40,351.602,28020,Cash,Male,23,23327,"814,885","24,701"
4,10000015,2016-01-05,Pink Cab,ATLANTA GA,8.73,114.62,97.776,27182,Card,Male,33,8536,"814,885","24,701"


In [45]:
print(df.shape)
df.info()

(359392, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction ID      359392 non-null  int64         
 1   Date of Travel      359392 non-null  datetime64[ns]
 2   Company             359392 non-null  object        
 3   City                359392 non-null  object        
 4   KM Travelled        359392 non-null  float64       
 5   Price Charged       359392 non-null  float64       
 6   Cost of Trip        359392 non-null  float64       
 7   Customer ID         359392 non-null  int64         
 8   Payment_Mode        359392 non-null  object        
 9   Gender              359392 non-null  object        
 10  Age                 359392 non-null  int64         
 11  Income (USD/Month)  359392 non-null  int64         
 12  Population          359392 non-null  object        
 13  Users           

###**Exploratory Data Anlysis**

####**Data Cleaning**

In [46]:
def feature(i):
  #Extract the Year Column
  df['Year'] = pd.DatetimeIndex(df['Date of Travel']).year
  #Extract the Month column from Date of travel
  df['Month'] = pd.DatetimeIndex(df['Date of Travel']).month

  # Calculating profit
  df['Profit'] = df['Price Charged'] - df['Cost of Trip']

  #Extract day of the week
  df['Day_of_week'] = df['Date of Travel'].dt.day_name()
  #Drop population column
  #df.drop(columns=['Population'], inplace=True)

  #remove commas from users column
  df['Users']=df['Users'].str.replace(",", "", regex=False).astype(int)
  df['Population']=df['Population'].str.replace(",", "", regex=False).astype(int)

  return df
df=feature(df)
df.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip,Customer ID,Payment_Mode,Gender,Age,Income (USD/Month),Population,Users,Year,Month,Profit,Day_of_week
0,10000011,2016-01-10,Pink Cab,ATLANTA GA,30.45,370.95,313.635,29290,Card,Male,28,10813,814885,24701,2016,1,57.315,Sunday
1,10000012,2016-01-08,Pink Cab,ATLANTA GA,28.62,358.52,334.854,27703,Card,Male,27,9237,814885,24701,2016,1,23.666,Friday
2,10000013,2016-01-04,Pink Cab,ATLANTA GA,9.04,125.20,97.632,28712,Cash,Male,53,11242,814885,24701,2016,1,27.568,Monday
3,10000014,2016-01-09,Pink Cab,ATLANTA GA,33.17,377.40,351.602,28020,Cash,Male,23,23327,814885,24701,2016,1,25.798,Saturday
4,10000015,2016-01-05,Pink Cab,ATLANTA GA,8.73,114.62,97.776,27182,Card,Male,33,8536,814885,24701,2016,1,16.844,Tuesday


##**Comaprision Between Pink Cab and Yellow Cab Trends**

### Market Share

In [47]:
cab_color={'Pink Cab':'#ff66cc',
           'Yellow Cab':'#ffff00'}


# i=df['Company'].value_counts()
# print(i)
i=df.groupby('Company')['Transaction ID'].count().reset_index()
fig = px.pie(i,
             names="Company", values='Transaction ID',
             color='Company',
             color_discrete_map=cab_color,
             title="Share of cab rides between Yellow Cab and Pink Cab"
             )
fig.update_layout(title_x=0.5, width=500,height=500)
fig.show()

- Yellow cabs hold a dominant market share of 76.4%, compared to Pink Cabs' 23.6%.

In [48]:
year=df.groupby(['Year', 'Company'])['Transaction ID'].count().reset_index()
fig=px.bar(year, x='Year', y="Transaction ID", color='Company', barmode="group", color_discrete_map=cab_color, text_auto=True, title="Cab ride trends Over the years")
fig.show()

- Cab usage for both companies rose from 2016 to 2017. However, 2018 data shows a slight decline.However, information for 2019 is incomplete.

In [49]:
pop=df.groupby(["City"]).agg({'Users':pd.Series.mean,
                          'Population':pd.Series.mean,
                          }).reset_index()
pop['User/Pop']=round(((pop['Users']/pop['Population'])*100), 2)

pop.sort_values(by='Population', ascending=False, inplace=True)
pop

,City,Users,Population,User/Pop
9,NEW YORK NY,302149.0,8405837.0,3.59
3,CHICAGO IL,164468.0,1955130.0,8.41
6,LOS ANGELES CA,144132.0,1595037.0,9.04
7,MIAMI FL,17675.0,1339155.0,1.32
16,SILICON VALLEY,27247.0,1177609.0,2.31
10,ORANGE COUNTY,12994.0,1030185.0,1.26
14,SAN DIEGO CA,69995.0,959307.0,7.30
11,PHOENIX AZ,6133.0,943999.0,0.65
4,DALLAS TX,22157.0,942908.0,2.35
0,ATLANTA GA,24701.0,814885.0,3.03


In [50]:
fig=px.bar(pop, x='City', y='User/Pop',color_discrete_map=cab_color, title="Percentage of Cab Users per City", text_auto=True)
fig.update_layout(title_x=0.5, yaxis_title='Percentage of Cab Users')
fig.show()

- Surprisingly, New York City, despite its large population, has a cab user rate of only 3.59%. In contrast, Boston and Washington boast significantly higher usage rates of 32.14% and 30.32%, respectively despite being less populus cities.

In [51]:
def two_var(x,y, z):
  i=df.groupby([x, y]).agg({z:pd.Series.nunique}).reset_index().sort_values(by=z, ascending=False)

  return i

city=two_var('City','Company', 'Transaction ID')
fig=px.bar(city, x='City', y='Transaction ID', color='Company',color_discrete_map=cab_color, title="Cab Market Share in Different Cities", text_auto=True )
fig.update_layout(title_x=0.5)
fig.show()

- Yellow Cab dominates the taxi market in most cities, except San Diego, Nashville, and Sacramento, where Pink Cab takes the lead.

In [52]:
#Create a plotly bar graph function to avoid repetition
def plot_bar(data, x, y, t):
  fig=px.bar(data, x, y, color='Company',color_discrete_map=cab_color, title=t, text_auto=True, barmode="group")
  fig.update_layout(title_x=0.5, yaxis_title="Total Number of Rides")
  return fig


In [53]:
g=df.groupby(["Gender"])['Transaction ID'].count().reset_index()
fig = px.pie(g, names="Gender",
             color='Gender',
             values='Transaction ID',
             title="Share of cab rides between Yellow Cab and Pink Cab"
             )
fig.update_layout(title_x=0.5, width=500,height=500)
fig.show()

In [54]:
gender=two_var("Company", 'Gender', 'Transaction ID')
fig=plot_bar(gender, x='Gender', y='Transaction ID', t="Cab Company prefence by Gender")
fig.update_layout(title_x=0.5, width=500,height=500)
fig.show();

- Data indicate men order taxis at a significantly higher rate at 57.3%. However, Yellow cabs seem to be popular with both genders, this is possibly due to it's wider availability compared to Pink Cab.

### Demographic Distribution

In [ ]:
# fig = px.violin(df, y="Income (USD/Month)", color="Gender", x="Company", color_discrete_map=cab_color, title="Income Distribution Between Gender and Company", box=True)
# fig.update_layout(title_x=0.5, width=800,height=500)
# fig.show();

- The violin plot reveals a close resemblance in income distribution between customers using both taxi services, regardless of gender.

In [ ]:
# fig = px.violin(df, y="Age", color="Gender", x="Company", color_discrete_map=cab_color, title="Age Distribution Between Gender and Company",  box=True)
# fig.update_layout(title_x=0.5, width=800,height=500)
# fig.show();

- Customers aged 20 to 40 dominate ridership for both cab services, with no significant difference in age distribution between the two companies.

In [ ]:
# p_correlation = df['KM Travelled'].corr(df['Profit'])
# print(p_correlation)

- Correlation Between Distance and Profit
- Prefence Between Gender,Age, city and Income class,
- Trends in Years, Months and Day of week

In [ ]:
# # New vs Existing Users
# df_first_Travel=df.groupby(["Customer ID"])["Date of Travel"].min().reset_index()
# df_first_Travel.columns=["Customer ID", "FirstTravelDate"]
# df=pd.merge(df, df_first_Travel, on="Customer ID")
# df["UserType"]="New"
# df.loc[df["Date of Travel"]>df["FirstTravelDate"], "UserType"]="Existing"

# df.head()



In [ ]:
# # New vs Existing User Revenue Analysis
# df_new_Profit=df.groupby(["Month", "Year", "UserType"])["Profit"].sum().reset_index()
# fig_dims=(6,6)
# plt.figure()
# sns.relplot(x="Month", y="Profit", hue="UserType", data=df_new_Profit, kind="line")
# plt.title("New vs Existing Customer Profit Overview")
# plt.xlabel("Month")
# plt.ylabel("Profit");

In [ ]:
# # Recency Calculation
# df_user=pd.DataFrame(df["Customer ID"].unique())
# df_user.columns=["Customer ID"]
# df_last_travel=df.groupby(["Customer ID"])["Date of Travel", "Company"].max().reset_index()
# df_last_travel.columns=["Customer ID", "LastTravelDate","Company"]
# df_last_travel["Recency"]= (df_last_travel["LastTravelDate"].max()-df_last_travel["LastTravelDate"]).dt.days
# df_recency=pd.merge(df_user, df_last_travel[["Customer ID", "Recency", "Company"]])
# df_recency.head()

In [ ]:
# # Look at the distribution of Recency
# plt.figure(figsize=(15,10))
# #sns.distplot(df_recency["Recency"], hue="Company")
# sns.displot(df_recency, x="Recency", hue="Company", kind="kde", fill=True)
# plt.title("Recency Distribution Within the Customers")
# plt.xlabel("Recency")
# plt.ylabel("Customer Count");

In [ ]:
# df_recency.to_csv('Recency')
# files.download('NewvsExisting.csv')

In [ ]:
# # frequency of orders
# df_frequency=df.groupby(["Company", "Customer ID"])["Date of Travel"].count().reset_index()
# df_frequency.columns=["Company", "Customer ID", "Frequency"]
# df_frequency=pd.merge(df_user, df_frequency, on="Customer ID")
# df_frequency.head()

In [ ]:
# df_frequency.to_csv('frequency.csv')
# files.download('frequency.csv')

In [ ]:
# # Review of Frequency Distribution
# plt.figure(figsize=(7,5))
# #sns.distplot(df_frequency, x="Frequency", hue="Company")
# sns.displot(df_frequency, x="Frequency", hue="Company", kind="kde", fill=True)
# plt.title("Frequency Distribution")
# plt.xlabel("Frequency")
# plt.ylabel("Count")

In [ ]:
# fig = px.histogram(df_frequency, x="Frequency", color="Company", title="Frequency Distribution")
# fig.show()

In [ ]:
# df_customer_revenue=df.groupby(["Company", "Customer ID"])["Profit"].sum().reset_index()
# df_customer_revenue=pd.merge(df_user, df_customer_revenue, on="Customer ID")
# df_customer_revenue.head()


In [ ]:
# # Revenue Distribution
# plt.figure(figsize=(15,10))
# #sns.distplot(df_customer_revenue.query("Profit < 10000")["Profit"])
# #sns.displot(df_customer_revenue, x="Profit", hue="Company", kind="kde", fill=True);
# fig = px.histogram(df_customer_revenue, x="Profit", color="Company", title="Profit Distribution")
# fig.show()

In [ ]:
# #Peak days that people hail Cabs
# days = [day for day, df in df.groupby('Day_of_week')]
# #plt.plot(days, df.groupby(['Day_of_week']).count());
# sns.lineplot(data=df, x="Day_of_week", y="Profit", hue="Company");

In [ ]:
# #customer segmentation
# sns.catplot(x="Gender", y="Profit", hue="Company", kind="bar", data=df);